<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Project-description" data-toc-modified-id="Project-description-1">Project description</a></span></li><li><span><a href="#Data-Preparation" data-toc-modified-id="Data-Preparation-2">Data Preparation</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Useless-Columns" data-toc-modified-id="Useless-Columns-2.0.0.1">Useless Columns</a></span></li><li><span><a href="#Missing-Values" data-toc-modified-id="Missing-Values-2.0.0.2">Missing Values</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Models-Selection" data-toc-modified-id="Models-Selection-3">Models Selection</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-3.0.1">CatBoost</a></span><ul class="toc-item"><li><span><a href="#Finding-Best-Hyper-Parameters-With-Grid-Search" data-toc-modified-id="Finding-Best-Hyper-Parameters-With-Grid-Search-3.0.1.1">Finding Best Hyper-Parameters With Grid Search</a></span></li><li><span><a href="#CatBoost-With-Best-Hyper-Parameters" data-toc-modified-id="CatBoost-With-Best-Hyper-Parameters-3.0.1.2">CatBoost With Best Hyper-Parameters</a></span></li></ul></li><li><span><a href="#LightGMB:-LGBMRegressor" data-toc-modified-id="LightGMB:-LGBMRegressor-3.0.2">LightGMB: LGBMRegressor</a></span><ul class="toc-item"><li><span><a href="#Finding-Best-Hyper-Parameters-With-Grid-Search" data-toc-modified-id="Finding-Best-Hyper-Parameters-With-Grid-Search-3.0.2.1">Finding Best Hyper-Parameters With Grid Search</a></span></li><li><span><a href="#LGBMRegressor-With-Best-Hyper-Parameters" data-toc-modified-id="LGBMRegressor-With-Best-Hyper-Parameters-3.0.2.2">LGBMRegressor With Best Hyper-Parameters</a></span></li></ul></li><li><span><a href="#XGBoost" data-toc-modified-id="XGBoost-3.0.3">XGBoost</a></span></li></ul></li><li><span><a href="#Dummy-model" data-toc-modified-id="Dummy-model-3.1">Dummy model</a></span></li></ul></li><li><span><a href="#Models-Comparison" data-toc-modified-id="Models-Comparison-4">Models Comparison</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-5">Conclusion</a></span></li></ul></div>

# Project description

Rusty Bargin used car sales service is developing an app to attract new customers. In that app, a user can quickly find out the market value of their car.<br>
<b>The task</b> is to develop a model to determine the value of a car.<br>
<b>Data description</b>
Historic data was provided and it includes: technical specifications, trim versions and prices. The data description:<br>
<i>DateCrawled</i> — date a profile was downloaded from the database<br>
<i>VehicleType</i> — vehicle body type<br>
<i>RegistraionYear</i> — vehicle registration year<br>
<i>Gearbox</i> — gearbox type<br>
<i>Power</i> — power (hp)<br>
<i>Model</i> — vehicle model<br>
<i>Mileage</i> — mileage (measured in km)<br>
<i>RegistrationMonth</i> — vehicle registration month<br>
<i>FuelType</i> — fuel type<br>
<i>Brand</i> — vehicle brand<br>
<i>NotRepaired</i> — vehicle repaired or not<br>
<i>DateCreated</i> — date of the profile creation<br>
<i>NumberOfPictures</i> — number of vehicle pictures<br>
<i>PostalCode</i> — postal code of the profile owner (user)<br>
<i>LastSeen</i> — date of the last activity of the user<br>

# Data Preparation

In [1]:
import sys
import warnings
if not sys.warnoptions:
       warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import xgboost as xgb

In [2]:
df = pd.read_csv('car_data.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
df.sample(2)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
159269,07/04/2016 00:06,7500,small,2011,manual,60,ibiza,70000,8,petrol,seat,no,06/04/2016 00:00,0,57520,07/04/2016 01:25
34859,08/03/2016 12:54,100,small,1998,manual,50,fiesta,150000,3,petrol,ford,no,08/03/2016 00:00,0,41464,15/03/2016 09:15


In [5]:
df.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


#### Useless Columns

All columns with dates are not really predictors for this task, so let's just remove them. Also, the column with number of pictures has all zeros. Perhaps this happened due to error while data dump. We will remove it too.

In [6]:
df.drop(['DateCrawled', 'DateCreated', 'LastSeen', 'NumberOfPictures'], axis=1, inplace=True)

#### Missing Values

In [7]:
def count_missing_by_col(df):
    return df.isna().sum().to_frame(name="# of missing values ('NaNs')").T

count_missing_by_col(df)

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,PostalCode
# of missing values ('NaNs'),0,37490,0,19833,0,19705,0,0,32895,0,71154,0


In [8]:
# % of missing data in each column
round(df.isnull().sum() / df.shape[0] * 100, 2).to_frame(name="% of missing values ('NaNs')").T

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,PostalCode
% of missing values ('NaNs'),0.0,10.58,0.0,5.6,0.0,5.56,0.0,0.0,9.28,0.0,20.08,0.0


There are several methods to deal with missing values. In this project we will fill all missing categorical values with "Nan", e.g. we will create a separate category for them.

In [9]:
categorical = list(df.loc[:, df.dtypes == 'object'].columns.values)
categorical

['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']

In [10]:
df[categorical] = df[categorical].fillna('Nan')

# Models Selection

We will split the data into training, validation and test sets in the following ratio: 3:1:1 respectively.

In [11]:
# Splitting into 80% and 20%
remainder, test = train_test_split(df, test_size=0.2, random_state=1)
# splitting 80% into 75% and 25%, thus getting 60% and 20% of total respectively
train, valid = train_test_split(remainder, test_size=0.25, random_state=1)
print(train.shape, test.shape, valid.shape)

(212621, 12) (70874, 12) (70874, 12)


In [12]:
X_train = train.drop(['Price'], axis=1)
X_test = test.drop(['Price'], axis=1)
X_valid = valid.drop(['Price'], axis=1)
y_train = train['Price']
y_test = test['Price']
y_valid = valid['Price']

### CatBoost

We do not have to encode categorical values for the CatBoost model, but we need to indicate what features are categorical. Also, CatBoost will not work with missing data, but we already took care of it.

#### Finding Best Hyper-Parameters With Grid Search

In [13]:
catboost = CatBoostRegressor(cat_features = categorical)

In [14]:
param_grid = {'n_estimators': [100],
        'learning_rate': [0.05, 0.1, 0.15],
        'depth': [6, 8, 10],
        #'random_state': [123],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

In [15]:
grid_search_result = catboost.grid_search(param_grid, X=X_train, y=y_train) #, plot=True) #cv=3)

0:	learn: 6051.4103172	test: 6069.5802599	best: 6069.5802599 (0)	total: 80.2ms	remaining: 7.94s
1:	learn: 5811.1217956	test: 5831.0038673	best: 5831.0038673 (1)	total: 105ms	remaining: 5.16s
2:	learn: 5589.0546955	test: 5609.6450777	best: 5609.6450777 (2)	total: 132ms	remaining: 4.28s
3:	learn: 5378.3155964	test: 5400.9154081	best: 5400.9154081 (3)	total: 152ms	remaining: 3.66s
4:	learn: 5176.8035263	test: 5200.3402741	best: 5200.3402741 (4)	total: 174ms	remaining: 3.3s
5:	learn: 4987.9454866	test: 5012.3913124	best: 5012.3913124 (5)	total: 194ms	remaining: 3.04s
6:	learn: 4809.9297327	test: 4834.7215082	best: 4834.7215082 (6)	total: 215ms	remaining: 2.86s
7:	learn: 4643.0798567	test: 4668.5346486	best: 4668.5346486 (7)	total: 236ms	remaining: 2.71s
8:	learn: 4485.9455689	test: 4512.4448960	best: 4512.4448960 (8)	total: 257ms	remaining: 2.6s
9:	learn: 4339.6109577	test: 4366.9943334	best: 4366.9943334 (9)	total: 281ms	remaining: 2.52s
10:	learn: 4201.4095914	test: 4228.9488421	best: 42

90:	learn: 2031.7079128	test: 2065.1211583	best: 2065.1211583 (90)	total: 2.04s	remaining: 202ms
91:	learn: 2029.7743759	test: 2063.1060646	best: 2063.1060646 (91)	total: 2.06s	remaining: 179ms
92:	learn: 2027.8577913	test: 2061.0824824	best: 2061.0824824 (92)	total: 2.08s	remaining: 157ms
93:	learn: 2025.8970243	test: 2059.5160444	best: 2059.5160444 (93)	total: 2.1s	remaining: 134ms
94:	learn: 2024.5857467	test: 2058.2660897	best: 2058.2660897 (94)	total: 2.12s	remaining: 111ms
95:	learn: 2022.7669289	test: 2056.4729228	best: 2056.4729228 (95)	total: 2.13s	remaining: 88.8ms
96:	learn: 2021.0885074	test: 2054.7881255	best: 2054.7881255 (96)	total: 2.15s	remaining: 66.5ms
97:	learn: 2019.3653647	test: 2053.1440069	best: 2053.1440069 (97)	total: 2.16s	remaining: 44.2ms
98:	learn: 2017.1989037	test: 2051.0844777	best: 2051.0844777 (98)	total: 2.18s	remaining: 22.1ms
99:	learn: 2015.1953958	test: 2049.0240336	best: 2049.0240336 (99)	total: 2.2s	remaining: 0us

bestTest = 2049.024034
bestIt

limit_output extension: Maximum message size of 10000 exceeded with 10547 characters

In [16]:
grid_search_result['params']

{'depth': 10, 'l2_leaf_reg': 1, 'iterations': 100, 'learning_rate': 0.15}

#### CatBoost With Best Hyper-Parameters

In [17]:
catboost_best = CatBoostRegressor(iterations=100, 
                                  learning_rate=0.15, 
                                  depth=10,
                                  l2_leaf_reg= 1,
                                  random_seed=123, 
                                  cat_features=categorical)

In [18]:
%%time
catboost_best.fit(X_train, y_train, cat_features=categorical, verbose=100)

0:	learn: 4051.3871512	total: 39.5ms	remaining: 3.91s
99:	learn: 1684.1539358	total: 3.64s	remaining: 0us
CPU times: user 20.1 s, sys: 1.32 s, total: 21.4 s
Wall time: 4.06 s


In [19]:
%%time
catboost_predictions_valid = catboost_best.predict(X_valid)

CPU times: user 165 ms, sys: 5.61 ms, total: 170 ms
Wall time: 104 ms


In [20]:
rmse_cat_valid = mean_squared_error(y_valid, catboost_predictions_valid)**0.5
print("RMSE on validation set: %.2f" % rmse_cat_valid)

RMSE on validation set: 1811.47


In [21]:
rmse_cat_test = mean_squared_error(y_test, catboost_best.predict(X_test))**0.5
print("RMSE on validation set: %.2f" % rmse_cat_test)

RMSE on validation set: 1790.44


### LightGMB: LGBMRegressor

Another model that we will try is LGBMRegressor. Similar to CatBoost this model can handle categorical values without having to encode them, but we do need to change the type of categorical values from "object" to "category". Let's prepare features for LGBMRegressor. 

In [22]:
categorical

['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']

In [23]:
for feature in categorical:
    X_train[feature] = pd.Series(X_train[feature], dtype="category")
    X_valid[feature] = pd.Series(X_valid[feature], dtype="category")
    X_test[feature] = pd.Series(X_test[feature], dtype="category")

#### Finding Best Hyper-Parameters With Grid Search

In [24]:
lgbm_model = LGBMRegressor()

In [25]:
param_grid = {
    'learning_rate': [0.05, 0.1, 0.15],
    'n_estimators': [100, 200, 300, 400], #Number of boosted trees to fit.
    'max_depth': [5, 10, 15, 20, 25] #Maximum tree depth for base learners, <=0 means no limit.
}

In [26]:
grid_search_lgbm = GridSearchCV(estimator=lgbm_model,
                                param_grid=param_grid,
                                cv=4,
                                scoring='neg_root_mean_squared_error')

In [27]:
grid_search_lgbm.fit(X_train, y_train, categorical_feature=categorical)

GridSearchCV(cv=4, estimator=LGBMRegressor(),
             param_grid={'learning_rate': [0.05, 0.1, 0.15],
                         'max_depth': [5, 10, 15, 20, 25],
                         'n_estimators': [100, 200, 300, 400]},
             scoring='neg_root_mean_squared_error')

In [28]:
grid_search_lgbm.best_params_

{'learning_rate': 0.15, 'max_depth': 15, 'n_estimators': 400}

#### LGBMRegressor With Best Hyper-Parameters

In [29]:
lgbm_best = LGBMRegressor(learning_rate=0.15, max_depth=15, n_estimators=400)

In [30]:
%%time
lgbm_best.fit(X_train, y_train, categorical_feature=categorical) 

CPU times: user 10.3 s, sys: 342 ms, total: 10.7 s
Wall time: 1.64 s


LGBMRegressor(learning_rate=0.15, max_depth=15, n_estimators=400)

In [31]:
%%time
lgbm_predictions_valid = lgbm_best.predict(X_valid)

CPU times: user 2.13 s, sys: 42 ms, total: 2.17 s
Wall time: 371 ms


In [32]:
rmse_lgbm_valid = mean_squared_error(y_valid, lgbm_predictions_valid)**0.5
print("RMSE on validation set: %.2f" % rmse_lgbm_valid)

RMSE on validation set: 1739.70


In [33]:
rmse_lgbm_test = mean_squared_error(y_test, lgbm_best.predict(X_test))**0.5
print("RMSE on validation set: %.2f" % rmse_lgbm_test)

RMSE on validation set: 1716.87


### XGBoost

XGBBoost requires categorical values to be encoded. 

In [34]:
categorical

['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']

In [35]:
label_encoder = LabelEncoder()

for col in categorical:
    X_train[col] = label_encoder.fit_transform(X_train[col])
    X_test[col] = label_encoder.fit_transform(X_test[col])
    X_valid[col] = label_encoder.fit_transform(X_valid[col])

In [36]:
dtrain = xgb.DMatrix(data=X_train, label=y_train)
dvalid= xgb.DMatrix(data=X_valid, label=y_valid)
dtest=xgb.DMatrix(data=X_test, label=y_test)

In [37]:
params = {'objective':'reg:linear', 'colsample_bytree': 0.3, 'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10, 'n_estimators': 10, 'random_state':123}

In [38]:
%%time
bst = xgb.train(params, dtrain)

[21:18:34] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[21:18:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "n_estimators" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


CPU times: user 1.88 s, sys: 97.3 ms, total: 1.98 s
Wall time: 308 ms


In [39]:
%%time
xgb_predictions_valid = bst.predict(dvalid)

CPU times: user 27.3 ms, sys: 2.21 ms, total: 29.5 ms
Wall time: 4.52 ms


In [40]:
rmse_xgb_valid = mean_squared_error(y_valid, xgb_predictions_valid)**0.5
print("RMSE validation set: %.2f" % rmse_xgb_valid)

RMSE validation set: 3834.46


In [41]:
rmse_xgb_test = round(mean_squared_error(y_test, bst.predict(dtest))**0.5, 0)
print("RMSE test set: %.2f" % rmse_xgb_test)

RMSE test set: 3833.00


## Dummy model

In [42]:
mean = y_train.mean()
length = X_test.shape[0]
dummy_predictions = np.full((length,1), mean)
dummy_rmse = mean_squared_error(dummy_predictions, y_test)**0.5
print('RMSE dummy model:',dummy_rmse)

RMSE dummy model: 4519.468512614231


# Models Comparison

Among the three models that we have tried the best accuracy was achieved with the Random Forest Classifier model with the number of trees in the forest set to 25.

In [43]:
d = {'RSME_valid': [rmse_cat_valid, rmse_lgbm_valid, rmse_xgb_valid, dummy_rmse],
     'RSME_test': [rmse_cat_test, rmse_lgbm_test, rmse_xgb_test, dummy_rmse],
     'Training time': ['3-4 s', '1-1.1 s', '290-330 ms', '--'],
     'Prediction time': ['110-140 ms', '290-300 ms', '5-9 ms', '--']} 
     
final = pd.DataFrame(data=d, index = ['CatBoostRegressor', 'LGBMRegressor', 'XGb',  'Dummy Model'])
display(final)

,RSME_valid,RSME_test,Training time,Prediction time
CatBoostRegressor,1811.468574,1790.444140,3-4 s,110-140 ms
LGBMRegressor,1739.699177,1716.868107,1-1.1 s,290-300 ms
XGb,3834.456752,3833.000000,290-330 ms,5-9 ms
Dummy Model,4519.468513,4519.468513,--,--


Among all three models LGBMRegressor demonstrates best accuracy and second least run time for training and prediction. XGB model is seen to be least accurate: it does a bit better than a dummy model, which just predicts the mean value. Plus XGB model's run time is the highest among all.

# Conclusion

 1. We looked into the given data and found some missing values. All missing values were categorical. We chose to deal with missing categorical values by creating a new category 'Nan'.<br>
 2. We compared three models and chose the best model - LGBMRegressor, whose RMSE metric is the lowest, meaning the best. Also, it demonstrates good run time in terms of speed of prediction.<br>
The CatBoostRegressor is the runner up. It also demonstrated very good RMSE metric and acceptable time.